## Step 1: Verify GPU & Environment

In [ ]:
import torch
import os
from pathlib import Path

print("="*70)
print("COLAB ENVIRONMENT CHECK")
print("="*70)

print(f"\nPython: {os.sys.version.split()[0]}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print(f"\n✅ GPU is ready!")
else:
    print("\n⚠️  No GPU detected. Enable GPU:")
    print("   Runtime → Change runtime type → GPU (T4)")

print("\n" + "="*70)

## Step 2: Clone Repository from GitHub

In [ ]:
import subprocess
from pathlib import Path

REPO_URL = "https://github.com/wwebtvmedia/enhancedlibcem.git"
REPO_PATH = Path("/content/enhancedlibcem")

print("Cloning repository...")

if REPO_PATH.exists():
    print(f"Repository already exists at {REPO_PATH}")
    # Update if already cloned
    subprocess.run(["git", "-C", str(REPO_PATH), "pull"], check=False)
else:
    subprocess.run(["git", "clone", REPO_URL, str(REPO_PATH)], check=True)

# List contents
files = sorted(list(REPO_PATH.glob("*")))
print(f"\n✅ Repository ready. Contents ({len(files)} items):")
for f in files[:15]:  # Show first 15
    ftype = "[DIR]" if f.is_dir() else f"[{f.suffix}]"
    print(f"  {ftype:6} {f.name}")

if len(files) > 15:
    print(f"  ... and {len(files)-15} more files")

## Step 3: Mount Google Drive (Optional - for saving results)

In [ ]:
from google.colab import drive
from pathlib import Path

try:
    drive.mount('/content/drive', force_remount=False)
    print("✅ Google Drive mounted at /content/drive")
    
    # Create results folder
    results_dir = Path('/content/drive/MyDrive/enhancedlibcem_results')
    results_dir.mkdir(exist_ok=True)
    print(f"✅ Results folder created: {results_dir}")
except Exception as e:
    print(f"⚠️  Drive mount skipped: {e}")
    print("   You can still save results to /content/ and download manually")

## Step 4: Install Dependencies

In [ ]:
print("Installing PyTorch with CUDA support...")
import subprocess
import sys

# PyTorch (CUDA 11.8 compatible with Colab)
result = subprocess.run(
    [sys.executable, "-m", "pip", "install", "-q",
     "torch", "torchvision", "torchaudio",
     "--index-url", "https://download.pytorch.org/whl/cu118"],
    capture_output=True, text=True
)

if result.returncode == 0:
    print("✅ PyTorch installed")
else:
    print(f"⚠️  PyTorch install had warnings (OK): {result.stderr[:200]}")

In [ ]:
print("Installing CLIP and other dependencies...")
import subprocess
import sys

packages = [
    "git+https://github.com/openai/CLIP.git",
    "numpy",
    "matplotlib",
    "scikit-image",
    "scipy",
    "numba",
    "Pillow",
    "tqdm"
]

for pkg in packages:
    result = subprocess.run(
        [sys.executable, "-m", "pip", "install", "-q", pkg],
        capture_output=True, text=True, timeout=60
    )
    status = "✅" if result.returncode == 0 else "⚠️"
    pkg_name = pkg.split("/")[-1].split(".")[0] if "/" in pkg else pkg.split("=")[0]
    print(f"{status} {pkg_name}")

print("\n✅ All dependencies installed!")

## Step 5: Verify Checkpoint & Setup

In [ ]:
from pathlib import Path
import os

print("="*70)
print("CHECKING SETUP")
print("="*70)

repo_path = Path("/content/enhancedlibcem")
checkpoint_path = repo_path / "improved_lidecm.pt"

# Check repo
essential_files = [
    "enhancedlibcem.py",
    "quick_test.py",
    "inference_from_checkpoint.py",
    "improved_lidecm.pt"
]

print("\nEssential files:")
for fname in essential_files:
    fpath = repo_path / fname
    if fpath.exists():
        size = fpath.stat().st_size if fpath.is_file() else "dir"
        if isinstance(size, int):
            size = f"{size / 1e6:.1f} MB" if size > 1e6 else f"{size / 1e3:.1f} KB"
        print(f"  ✅ {fname:35} ({size})")
    else:
        print(f"  ❌ {fname:35} (MISSING)")

if checkpoint_path.exists():
    print(f"\n✅ Checkpoint ready for inference")
else:
    print(f"\n⚠️  Checkpoint not found. quick_test.py will still work.")
    print(f"   Checkpoint path: {checkpoint_path}")

print("\n" + "="*70)

## Step 6: Run Quick Diagnostic Test

In [ ]:
import os
import subprocess

os.chdir("/content/enhancedlibcem")

print("\n" + "="*70)
print("RUNNING DIAGNOSTIC TEST (quick_test.py)")
print("="*70 + "\n")

result = subprocess.run(
    ["python", "quick_test.py"],
    capture_output=False,
    text=True
)

print("\n" + "="*70)

In [ ]:
from pathlib import Path

diag_file = Path("/content/enhancedlibcem/quick_test_outputs/diagnostics.txt")

if diag_file.exists():
    print("\n" + "="*70)
    print("DIAGNOSTIC RESULTS")
    print("="*70)
    with open(diag_file, 'r') as f:
        print(f.read())
else:
    print("⚠️  Diagnostics file not found. Check quick_test.py output above.")

## Step 7: Display Diagnostic Images

In [ ]:
from IPython.display import Image, display
from pathlib import Path
import os

output_dir = Path("/content/enhancedlibcem/quick_test_outputs")

if output_dir.exists():
    print("\n" + "="*70)
    print("DIAGNOSTIC IMAGES")
    print("="*70)
    
    # Original images
    print("\n📷 Original Input Images:")
    orig_files = sorted(output_dir.glob("original_*.png"))
    for f in orig_files[:2]:
        print(f"\n{f.name}:")
        display(Image(str(f)))
    
    # Reconstructed images
    print("\n🔧 Model Reconstructed Images:")
    recon_files = sorted(output_dir.glob("reconstructed_*.png"))
    for f in recon_files[:2]:
        print(f"\n{f.name}:")
        display(Image(str(f)))
else:
    print("⚠️  Output directory not found.")

## Step 8: Run Inference with Checkpoint

In [ ]:
import os
import subprocess
from pathlib import Path

os.chdir("/content/enhancedlibcem")

# Ensure output dir exists
Path("/content/inference_out").mkdir(exist_ok=True)

print("\n" + "="*70)
print("RUNNING INFERENCE (inference_from_checkpoint.py)")
print("="*70 + "\n")

checkpoint_path = "/content/enhancedlibcem/improved_lidecm.pt"

if Path(checkpoint_path).exists():
    print(f"✅ Checkpoint found: {checkpoint_path}")
    cmd = ["python", "inference_from_checkpoint.py",
           "--checkpoint", checkpoint_path,
           "--outdir", "/content/inference_out"]
else:
    print(f"⚠️  Checkpoint not found at {checkpoint_path}")
    print("   Attempting inference anyway (may generate noise)...")
    cmd = ["python", "inference_from_checkpoint.py",
           "--outdir", "/content/inference_out"]

result = subprocess.run(cmd, capture_output=False, text=True)

print("\n" + "="*70)

## Step 9: Display Inference Results

In [ ]:
from IPython.display import Image, display
from pathlib import Path

inference_result = Path("/content/inference_out/inference_results.png")

if inference_result.exists():
    print("\n" + "="*70)
    print("🎨 INFERENCE RESULTS - TEXT-GUIDED IMAGE GENERATION")
    print("="*70)
    print(f"\n✅ Generated {inference_result.stat().st_size / 1e3:.1f} KB result image")
    print("\n(Shows 5 images generated from text prompts using the pre-trained model)\n")
    display(Image(str(inference_result)))
else:
    print("⚠️  inference_results.png not found.")
    print("   Check checkpoint and logs above.")

## Step 10: Save Results to Google Drive

In [ ]:
import shutil
from pathlib import Path

print("\n" + "="*70)
print("SAVING RESULTS TO GOOGLE DRIVE")
print("="*70 + "\n")

try:
    results_base = Path("/content/drive/MyDrive/enhancedlibcem_results")
    results_base.mkdir(exist_ok=True)
    
    # Copy quick_test outputs
    src = Path("/content/enhancedlibcem/quick_test_outputs")
    dst = results_base / "quick_test_outputs"
    if src.exists():
        shutil.copytree(src, dst, dirs_exist_ok=True)
        print(f"✅ Copied quick_test_outputs → {dst}")
    
    # Copy inference outputs
    src = Path("/content/inference_out")
    dst = results_base / "inference_out"
    if src.exists():
        shutil.copytree(src, dst, dirs_exist_ok=True)
        print(f"✅ Copied inference_out → {dst}")
    
    print(f"\n✅ All results saved to: /MyDrive/enhancedlibcem_results")
    print(f"   You can now download them from Google Drive")
    
except Exception as e:
    print(f"⚠️  Could not save to Drive: {e}")
    print("   Results are still available in /content/ (can download manually)")

## Summary & Next Steps

In [ ]:
from pathlib import Path
import json

print("\n" + "="*70)
print("✅ COLAB EXECUTION COMPLETED")
print("="*70)

print("\n📊 RESULTS SUMMARY:")
print()

# Check what was generated
quick_test_dir = Path("/content/enhancedlibcem/quick_test_outputs")
inference_dir = Path("/content/inference_out")

if quick_test_dir.exists():
    files = list(quick_test_dir.glob("*"))
    print(f"  Quick Test Outputs: {len(files)} files")
    for f in files:
        size = f.stat().st_size if f.is_file() else "dir"
        if isinstance(size, int):
            size = f"{size / 1e3:.1f} KB"
        print(f"    - {f.name} ({size})")

if inference_dir.exists():
    files = list(inference_dir.glob("*"))
    print(f"\n  Inference Outputs: {len(files)} files")
    for f in files:
        size = f.stat().st_size if f.is_file() else "dir"
        if isinstance(size, int):
            size = f"{size / 1e3:.1f} KB"
        print(f"    - {f.name} ({size})")

print("\n" + "="*70)
print("\n📌 NEXT STEPS:")
print()
print("  1. ✅ Diagnostics: Check if model converges (M-step loss, gradients)")
print("  2. ✅ Quality: Review original vs reconstructed images")
print("  3. ✅ Inference: View 5 text-guided generated images")
print("  4. 💾 Save: Results are in /MyDrive/enhancedlibcem_results")
print("  5. 🔧 Fine-tune: Modify prompts in inference_from_checkpoint.py")
print()
print("\n🔗 Code: https://github.com/wwebtvmedia/enhancedlibcem")
print("\n" + "="*70)

## Advanced: Custom Prompts (Optional)

In [ ]:
# To generate with custom prompts, modify this and run:
# IMPORTANT: Must run this AFTER Step 9 (inference already completed)

import os
os.chdir("/content/enhancedlibcem")

custom_prompts = [
    "a red car",
    "blue sky with clouds",
    "a cute dog",
    "abstract art",
    "mountain landscape"
]

print("To use custom prompts, run:")
print()
print("!python inference_from_checkpoint.py --checkpoint improved_lidecm.pt \\")
print("    --outdir /content/custom_out \\")
print(f"    --prompts {' '.join(f'\\"prompt\\"' for _ in custom_prompts)}")
print()
print("Replace 'prompt' with your desired text prompts.")